# Step5.3: get_VITfeats

运行它获得cross_modal_graph，为train做准备

书接上回，`dic`的格式如下：

```
{
    '682716753374351360': {
        'green wall': tensor([-0.2158, -0.5861, -0.4583,  ..., -0.4584, -0.0507, -0.4340]), 
        'thumb': tensor([-0.0943, -0.5097, -0.2850,  ..., -0.2975,  0.4255,  0.7431]), 
        'yellow circle': tensor([-0.3630,  0.0496, -0.5468,  ..., -0.5510, -0.0705, -0.4504]), 
        'blue lights': tensor([-0.4361, -0.5627, -0.6589,  ..., -0.6676, -0.3880, -0.6415]), 
        'clear glass': tensor([-0.3609, -0.6426, -0.6136,  ..., -0.6126, -0.5325, -0.5997]), 
        'hand': tensor([-0.4882, -0.5795, -0.6744,  ..., -0.6797, -0.4926, -0.5218]), 
        'blue dot': tensor([-0.0286, -0.0159, -0.2335,  ..., -0.2459, -0.5378,  0.2008]), 
        'man': tensor([-0.4034, -0.9045, -0.6479,  ..., -0.6523, -0.0525, -0.8093]), 
        'white words': tensor([-0.4899,  0.1618, -0.7007,  ..., -0.7047, -0.7450, -0.6402])
    }, 
    '682721949072625664': {
        'logo': tensor([ 0.1612,  0.3313, -0.0857,  ..., -0.1012,  0.2761,  0.1852]), 
        'white logo': tensor([ 0.0115, -0.3251, -0.1830,  ..., -0.1912,  0.7098, -0.2490]), 
        'blue sky': tensor([-0.1040,  0.2694, -0.2814,  ..., -0.2938, -0.1567,  0.1455]), 
        'white dog': tensor([-0.1709,  0.2743, -0.3800,  ..., -0.3780,  0.4276, -0.6060]), 
        'picture': tensor([ 0.1025,  0.3527, -0.0880,  ..., -0.0990,  0.7861, -0.1227]), 
        'blue wall': tensor([ 0.0800,  0.5311, -0.1281,  ..., -0.1368,  0.2102, -0.1506]), 
        'black letter': tensor([-0.2343, -0.0206, -0.4577,  ..., -0.4666,  0.2765,  0.2039]), 
        'green wall': tensor([-0.3552, -0.3162, -0.5775,  ..., -0.5958, -0.9602, -0.4342])
    }, 
    '682722242111774720': {
    ...
    },
}
```

In [1]:
import numpy as np
import spacy
import pickle
from tqdm import tqdm
from copy import deepcopy
from collections import defaultdict
from pytorch_pretrained import BertTokenizer
from nltk.corpus import wordnet as wn

tokenizer = BertTokenizer.from_pretrained("./bert-base-uncased")
nlp = spacy.load('en_core_web_sm')
y = 3
boxes = {}

block = 1

for i in range(block):
    with open(f"/root/autodl-tmp/vit_features.B32.finetuned.{i:02}.pkl",'rb') as fin:
        boxes_tmp = pickle.load(fin)
        boxes.update(boxes_tmp)
del boxes_tmp
print(len(boxes))

loading BertTokenizer.from_pretrained ./bert-base-uncased
os.path.exists(url_or_filename) ./bert-base-uncased/vocab.txt
2400


In [2]:
def dependency_adj_matrix(text, aspect, senticNet):
    word_list = text.split()
    seq_len = len(word_list)
    matrix = np.zeros((seq_len, seq_len)).astype('float32')
    
    for i in range(seq_len):
        word = word_list[i]
        if word in senticNet:
            sentic = float(senticNet[word]) + 1.0
        else:
            sentic = 0
        if word in aspect:
            sentic += 1.0
        for j in range(seq_len):
            matrix[i][j] += sentic
            matrix[j][i] += sentic
    for i in range(seq_len):
        if matrix[i][i] == 0:
            matrix[i][i] = 1

    return matrix

In [3]:
def dependency_adj_matrix_2(text):
    doc = nlp(text)
    mat = defaultdict(list,[])
    for t in doc:
        for child in t.children:
            mat[child.i].append(t.i)
            mat[t.i].append(child.i)
    return mat

In [4]:
# load senticNet load_sentic_word():

path = './senticNet/senticnet_word.txt'
senticNet = {}
fp = open(path, 'r')
for line in fp:
    line = line.strip()
    if not line:
        continue
    word, sentic = line.split('\t')
    senticNet[word] = float(sentic)
fp.close()

In [5]:
def get_sentic_score(word_i,word_j):
    if word_i not in senticNet or word_j not in senticNet or word_i == word_j:
        return 0
    return abs(float(senticNet[word_i] - senticNet[word_j])) * y**(-1*senticNet[word_i]*senticNet[word_j])

In [6]:
def generate_graph(line):
    line = line.lower().strip()
    bert_token = tokenizer.tokenize(line)
    document = nlp(line)
    spacy_token = [str(x) for x in document]
    
    # spacy_token = []
    # for x in document:
    #     x = str(x).strip()
    #     if x != '':
    #         spacy_token.append(x)
    
    spacy_len = len(spacy_token)
    bert_len = len(bert_token)
    outter_graph = np.zeros((bert_len, bert_len)).astype('float32')
    split_link = [None]*spacy_len
    
    # print(f"bert_token:  {bert_token}")
    # print(f"spacy_token: {spacy_token}")
    
    """
    bert_token:  ['we', 'must', 'rebuild', 'our', 'military', '!', '!', '!', 'we', 'need', 'more', 'battleships', '!', '#', 'go', '##pd', '##eb', '##ate']
    spacy_token: ['we', 'must', 'rebuild', 'our', 'military', '!', '!', '!', 'we', 'need', 'more', 'battleships', '!', '#', 'gopdebate']
    """
    
    # 这里应该是为了对齐分词 两种分词的方式是不一样的
    ii = 0
    jj = 0
    pre = []
    s = ""
    while ii<bert_len and jj < spacy_len:
        bert_ = bert_token[ii].replace("##","")
        spacy_ = spacy_token[jj]
        s += bert_
        pre.append(ii)
        if s == spacy_:
            split_link[jj] = deepcopy(pre)
            pre = []
            s = ""
            jj += 1        
        ii += 1
    
    # 暂时想不到别的办法了
    # 这种行不通的 长度变了 不能变长度
    # split_link = list(filter(lambda x: (x is not None) and (x != ""), split_link))
    # spacy_len = len(split_link)
    # bert_len = len(split_link)
    
    # for i in range(len(split_link)):
    #     if split_link[i] is None:
    #         split_link[i] = '[PAD]'
            
        
    flag = False
    
    mat = dependency_adj_matrix_2(line)
    if not(ii<bert_len or jj < spacy_len):
        for key,linked in mat.items():
            try:
                for x in split_link[int(key)]:
                    for link in linked:
                        for y in split_link[int(link)]:
                            outter_graph[x][y] = 1     
            except:
                flag = True
                break
    else:
        flag = True
        
    if flag:
        tokens = spacy_token
        outter_graph = np.zeros((spacy_len, spacy_len)).astype('float32')
        inner_graph = np.identity(spacy_len).astype('float32')
        doc = nlp(line)
        for token in doc:
            for child in token.children:
                outter_graph[token.i][child.i] = 1
                outter_graph[child.i][token.i] = 1    
    else:
        tokens = bert_token
        inner_graph = np.identity(bert_len).astype('float32')
        for link in split_link:
            for x in link:
                for y in link:
                    inner_graph[x][y] = 1

    
    outter_graph = np.pad(outter_graph,((1,1),(1,1)),'constant')
    inner_graph = np.pad(inner_graph,((1,1),(1,1)),'constant')
    inner_graph[0][0] = 1
    inner_graph[-1][-1] = 1
    graph_sum = inner_graph + outter_graph
    for i in range(len(graph_sum)):
        for j in range(len(graph_sum)):
            if graph_sum[i][j] > 0:
                graph_sum[i][j] = 1
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    return graph_sum,tokens,flag

# flag == True   ->  spacy_token 
# flag == False  ->  bert_token

In [7]:
def get_split(line):
    line = line.lower().strip()
    bert_token = tokenizer.tokenize(line)
    document = nlp(line)
    spacy_token = [str(x) for x in document]
    
    # spacy_token = []
    # for x in document:
    #     x = str(x).strip()
    #     if x != '':
    #         spacy_token.append(x)
    
    spacy_len = len(spacy_token)
    bert_len = len(bert_token)
    outter_graph = np.zeros((bert_len, bert_len)).astype('float32')
    split_link = [None]*spacy_len
    
    ii = 0
    jj = 0
    pre = []
    s = ""
    while ii < bert_len and jj < spacy_len:
        bert_ = bert_token[ii].replace("##","")
        spacy_ = spacy_token[jj]
        s += bert_
        pre.append(ii)
        if s == spacy_:
            split_link[jj] = deepcopy(pre)
            pre = []
            s = ""
            jj += 1        
        ii += 1
        
    mat = dependency_adj_matrix_2(line)
    if not(ii<bert_len or jj < spacy_len):
        for key,linked in mat.items():
            try:
                for x in split_link[int(key)]:
                    for link in linked:
                        for y in split_link[int(link)]:
                            outter_graph[x][y] = 1     
            except:
                break
    return split_link

In [8]:
def generate_image_graph(text,box_texts,flag):
    box_tokens = [list(generate_graph(text))+[text] for text in box_texts]
    box_tokens = [x[1:] for x in box_tokens]
    for idx,val in enumerate(box_tokens):
        box_tokens[idx] = [val[0][1:-1]] + val[1:]
    box_texts_len = sum([len(x[0]) for x in box_tokens])
    bert_text_tokens = tokenizer.tokenize(text.lower())
    spacy_text_tokens = nlp(text.lower())
    spacy_text_tokens = [str(x).lower() for x in spacy_text_tokens]

    flags = []
    word = []
    
    # flag == True   ->  spacy_token 
    # flag == False  ->  bert_token
    
    if flag:
        graph = np.zeros((len(spacy_text_tokens), (box_texts_len))).astype('float32')
        for i,token_i in enumerate(spacy_text_tokens):
            cur = 0
            si = wn.synsets(token_i)
            if len(si) == 0:
                continue
            si = si[0]
            for tokens,flag,text in box_tokens:
                flags.append(flag)
                if flag:
                    for j,token_j in enumerate(tokens):
                        sj = wn.synsets(token_j)
                        if i == 0:
                            word.append([token_j])
                        if len(sj) == 0:
                            cur += 1
                            continue
                        sj = sj[0]
                        graph[i][cur] = wn.path_similarity(si,sj) + get_sentic_score(si,sj)
                        cur += 1
                        
                else:
                    split_link1 = get_split(text)
                    tokens_ = nlp(text)
                    tokens_ = [str(x) for x in tokens_]
                    
                    for j,token_j in enumerate(tokens_):
                        sj = wn.synsets(token_j)
                        if i == 0:
                            tmp = []
                        for t in split_link1[j]:
                            if i == 0:
                                tmp.append(tokens[j])
                            if len(sj) == 0:
                                cur += 1
                                continue
                            sj_ = sj[0]
                            graph[i][cur] = wn.path_similarity(si,sj_) + get_sentic_score(si,sj_)
                            cur += 1
                        if i == 0:
                            word.append(tmp)
                        if len(sj) == 0:
                            continue

    else:
        graph = np.zeros((len(bert_text_tokens), (box_texts_len))).astype('float32')
        split_links_i_ = get_split(text)
        split_links_i = dict()
        for idx,value in enumerate(split_links_i_):
            for v in value:
                split_links_i[bert_text_tokens[v]] = spacy_text_tokens[idx]
        for i,token in enumerate(bert_text_tokens):
            cur = 0
            token = split_links_i[token]
            si = wn.synsets(token)
            if len(si) == 0:
                continue
            si = si[0]
            
            for tokens,flag,text in box_tokens:
                    flags.append(flag)
                    if flag:
                        for j,token_j in enumerate(tokens):
                            sj = wn.synsets(token_j)
                            if i == 0:
                                word.append([token_j])
                            if len(sj) == 0:
                                cur += 1
                                continue
                            sj = sj[0]
                            graph[i][cur] = wn.path_similarity(si,sj) + get_sentic_score(si,sj)
                            cur += 1            
                    else:
                        split_link1 = get_split(text)
                        tokens_ = nlp(text)
                        tokens_ = [str(x) for x in tokens_]
                        if i == 0:
                            tmp = []
                        for j,token_j in enumerate(tokens_):
                            sj = wn.synsets(token_j)
                            for t in split_link1[j]:
                                if i == 0:
                                    tmp.append(tokens[j])
                                if len(sj) == 0:
                                    cur += 1
                                    continue
                                sj_ = sj[0]
                                graph[i][cur] = wn.path_similarity(si,sj_) + get_sentic_score(si,sj_)
                                cur += 1
                                
                            if i == 0:
                                word.append(tmp)
                            if len(sj) == 0:
                                continue
    return np.pad(graph,((1,1),(1,1)),'constant'),flags,word

In [42]:
import cv2
import pandas as pd
images = pd.read_pickle("./bottom-up-attention/boxes.pkl")
  
def get_real_label(label):
    # [attr1 attr2 class1 class2]  -> [attr1 attr2_class1 class2]
    label_list = label.split()
    size = len(label_list)
    for i in range(size-1, 0,-1):
        if ' '.join(label_list[:i]) in attributes:
            if len(label_list[:i]) > 1:
                print(f"{' '.join(label_list[:i])}_{' '.join(label_list[i:])}")
            return f"{' '.join(label_list[:i])}_{' '.join(label_list[i:])}"
    else:
        # print("no attr")
        return label

In [43]:
# import json
# labels_map=json.load(open('labels_map.txt'))
# labels_map=[labels_map[str(i)] for i in range(1000)]
# labels_map

In [44]:
import os

data_path = './bottom-up-attention/data/genome/1600-400-20'

# Load classes
classes = ['__background__']
with open(os.path.join(data_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        classes.append(object.split(',')[0].lower().strip())

# Load attributes
attributes = ['__no_attribute__']
with open(os.path.join(data_path, 'attributes_vocab.txt')) as f:
    for att in f.readlines():
        attributes.append(att.split(',')[0].lower().strip())
        
classes_to_index = {}
for i in range(len(classes)):         
    classes_to_index[classes[i]] = i
len(classes_to_index)

1601

In [45]:
classes_to_index['wipers']

1114

In [46]:
import re
def keep_one_space(text):
    regex = re.compile(r"\s+")
    text = regex.sub(' ', text)
    return text

In [47]:
"""
['910308516510011393', 'most # funny quotes : 21 snarky and # funny quotes  # # funnyquotes # hilariousquotes # humor # lol ... <url> …', 1]
['725333760762363905', ' spurs # creativethinking ! <url>', 1]
['840006160660983809', '<user> thanks for showing up for our appointment today . ', 1]
"""

N = 10
# filename = "train.cleaned"
# outfile  = "processed_train_box_top{}".format(N)
def process(filename,outfile = ""):
    with open("./data/text/{}.txt".format(filename),'r',encoding='utf-8') as fin:
        lines = fin.readlines()
        lines = [x.strip() for x in lines]
        # lines = lines[:200]
        dic = dict()
        for i in tqdm(range(len(lines))):
            line = lines[i]
            data = eval(line)
            
            if 'train' in filename:
                id_, text, label = data
            else:
                id_, text, label1, label = data
            if id_ in boxes:
                # print(data)
                box_text = list(boxes[id_].items())
                
                box_text_tmp = []
                for key,val in box_text:
                    key = get_real_label(key)
                    box_text_tmp.append((" ".join(key.split("_")[:-1]), classes_to_index[key.split("_")[-1]], val))
                box_text = box_text_tmp
                del box_text_tmp
                
                box_text = sorted(box_text,key = lambda x:x[1])
                box_text = [(x,z) for x,y,z in box_text]

                box_text = box_text[:N]
                box_vit = [x[1][0] for x in box_text]
                
                text = keep_one_space(text) # 预处理，删除多余的空格，每个单词中间只留一个空格
                
                box_token = [tokenizer.tokenize(text.lower()) for text,_ in box_text]

                text_graph,tokens,flag = generate_graph(text.lower())
                image_graph,flags,word = generate_image_graph(text.lower(),[x[0] for x in box_text],flag)
                
                # flag == True   ->  spacy_token 
                # flag == False  ->  bert_token
                
                dic[id_] = {
                    'text':text,
                    'label':int(label),
                    'tokens':tokens,
                    'graph':text_graph,
                    'box_tokens':box_token,
                    'box_vit':box_vit,
                    'image_graph':image_graph, #value['graph'] + value["sentic_graph"]
                    "flags":flags,
                    "flag":flag,
                    'word':word,
                    "box_text":box_text
                }

                
                # print(text,tokens, word)
        print("{} : {} ".format(filename,len(dic)))
        pickle.dump(dic,open(f"{outfile}.data",'wb'))


process("train.cleaned","/root/autodl-tmp/processed_train_box_top{}".format(N))
process("test.cleaned","/root/autodl-tmp/processed_test_box_top{}".format(N))
process("valid.cleaned","/root/autodl-tmp/processed_valid_box_top{}".format(N))

100%|██████████| 29040/29040 [34:48<00:00, 13.91it/s]  


train.cleaned : 1970 


100%|██████████| 2409/2409 [03:38<00:00, 11.02it/s]


test.cleaned : 208 


100%|██████████| 2410/2410 [04:19<00:00,  9.28it/s] 


valid.cleaned : 222 


In [56]:
"""
['910308516510011393', 'most # funny quotes : 21 snarky and # funny quotes  # # funnyquotes # hilariousquotes # humor # lol ... <url> …', 1]
['725333760762363905', ' spurs # creativethinking ! <url>', 1]
['840006160660983809', '<user> thanks for showing up for our appointment today . ', 1]
"""

N = 10
# filename = "train.cleaned"
# outfile  = "processed_train_box_top{}".format(N)
def process(filename,outfile = ""):
    with open("./data/text/{}.txt".format(filename),'r',encoding='utf-8') as fin:
        lines = fin.readlines()
        lines = [x.strip() for x in lines]
        # lines = lines[:200]
        dic = dict()
        for i in tqdm(range(len(lines))):
            line = lines[i]
            data = eval(line)
            
            if 'train' in filename:
                id_, text, label = data
            else:
                id_, text, label1, label = data
            if id_ in boxes:
                # print(data)
                box_text = list(boxes[id_].items())

                box_text_tmp = []
                for key,val in box_text:
                    key = get_real_label(key)
                    box_text_tmp.append((" ".join(key.split("_")[:-1]), classes_to_index[key.split("_")[-1]], val))
                box_text = box_text_tmp
                del box_text_tmp
                
                box_text = sorted(box_text,key = lambda x:x[1])
                box_text = [(x,z) for x,y,z in box_text]

                box_text = box_text[:N]
                for x in box_text:
                    print(x[1])
                    print(x[1][0:1])
                box_vit = [x[1][0] for x in box_text]
                print(box_vit)
                
                text = keep_one_space(text) # 预处理，删除多余的空格，每个单词中间只留一个空格
                
                box_token = [tokenizer.tokenize(text.lower()) for text,_ in box_text]

                text_graph,tokens,flag = generate_graph(text.lower())
                image_graph,flags,word = generate_image_graph(text.lower(),[x[0] for x in box_text],flag)
                
                # flag == True   ->  spacy_token 
                # flag == False  ->  bert_token
                
                dic[id_] = {
                    'text':text,
                    'label':int(label),
                    'tokens':tokens,
                    'graph':text_graph,
                    'box_tokens':box_token,
                    'box_vit':box_vit,
                    'image_graph':image_graph, #value['graph'] + value["sentic_graph"]
                    "flags":flags,
                    "flag":flag,
                    'word':word,
                    "box_text":box_text
                }

                
                # print(text,tokens, word)
        print("{} : {} ".format(filename,len(dic)))
        pickle.dump(dic,open(f"{outfile}.data",'wb'))


process("train.cleaned","/root/autodl-tmp/processed_train_box_top{}111".format(N))
process("test.cleaned","/root/autodl-tmp/processed_test_box_top{}111".format(N))
process("valid.cleaned","/root/autodl-tmp/processed_valid_box_top{}111".format(N))

  0%|          | 0/29040 [00:00<?, ?it/s]

tensor([ 0.0389, -0.1254, -0.1880,  ..., -0.1860,  0.1868, -0.1006])
tensor([0.0389])
tensor([-0.2407, -0.0252, -0.4480,  ..., -0.4536,  0.1642, -1.1719])
tensor([-0.2407])
tensor([ 0.1094,  0.0883, -0.1254,  ..., -0.1236,  0.2345,  0.2256])
tensor([0.1094])
tensor([-0.4438,  0.1112, -0.6410,  ..., -0.6592, -0.2009, -1.4022])
tensor([-0.4438])
tensor([0.4070, 0.3969, 0.1934,  ..., 0.1958, 0.4265, 0.5157])
tensor([0.4070])
tensor([ 0.1299,  0.1064, -0.0962,  ..., -0.0941,  0.4523, -0.0670])
tensor([0.1299])
tensor([ 0.3860,  0.2687,  0.1727,  ...,  0.1713, -0.1640, -0.0461])
tensor([0.3860])
tensor([ 0.2770, -0.0342,  0.0583,  ...,  0.0515, -0.1309,  0.4865])
tensor([0.2770])
tensor([-0.2115, -0.1467, -0.3990,  ..., -0.3971, -0.1272, -0.1456])
tensor([-0.2115])
tensor([-0.3630, -0.3123, -0.5732,  ..., -0.5963, -0.0627, -0.8018])
tensor([-0.3630])
[tensor(0.0389), tensor(-0.2407), tensor(0.1094), tensor(-0.4438), tensor(0.4070), tensor(0.1299), tensor(0.3860), tensor(0.2770), tensor(-0.2

  0%|          | 12/29040 [00:02<1:37:21,  4.97it/s]

tensor([-0.1653,  0.1291, -0.4402,  ..., -0.4367,  0.1336, -0.4129])
tensor([-0.1653])
tensor([ 0.0162, -0.2315, -0.2202,  ..., -0.2361, -0.2290,  0.5811])
tensor([0.0162])
tensor([-0.0784, -0.2372, -0.2613,  ..., -0.2857, -0.2107, -0.5655])
tensor([-0.0784])
tensor([-0.1821, -0.4282, -0.3589,  ..., -0.3830,  0.0472, -0.0353])
tensor([-0.1821])
tensor([-0.2392, -0.5626, -0.4309,  ..., -0.4441, -0.3642, -0.3220])
tensor([-0.2392])
tensor([-0.0189,  0.2603, -0.2337,  ..., -0.2390,  0.0012,  0.2118])
tensor([-0.0189])
tensor([-0.0142,  0.0452, -0.1858,  ..., -0.2003,  0.0173, -0.5138])
tensor([-0.0142])
tensor([-0.2560, -0.4785, -0.4394,  ..., -0.4516, -1.1090, -0.5965])
tensor([-0.2560])
tensor([ 0.0486, -0.1447, -0.1712,  ..., -0.1961, -0.4140, -0.4611])
tensor([0.0486])
tensor([-0.1917, -0.4392, -0.3953,  ..., -0.4086, -0.3131,  0.5150])
tensor([-0.1917])
[tensor(-0.1653), tensor(0.0162), tensor(-0.0784), tensor(-0.1821), tensor(-0.2392), tensor(-0.0189), tensor(-0.0142), tensor(-0.256

  0%|          | 17/29040 [00:03<1:27:12,  5.55it/s]


KeyboardInterrupt: 

`dic`保存格式如下：

```
{
    '687826579599196160': {
        'text': 'we must rebuild our military ! ! ! we need more battleships ! # gopdebate', 
        'label': 1, 
        'tokens': [
                    '[CLS]', 'we', 'must', 'rebuild', 'our', 'military', 
                    '!', '!', '!', 'we', 'need', 'more', 'battleships', 
                    '!', '#', 'gopdebate', '[SEP]'
                ], 
        'graph': array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0., 0.],
                        [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0., 0.],
                        ...
                        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0., 1.]], 
                        dtype=float32),
        'box_tokens': [['black'], ['white'], ['white'], ['white'], ['blue'], ['black'], ['black']], 
        'box_vit': [
                    tensor(-0.2899), tensor(-0.2732), tensor(-0.3721), tensor(-0.2054), 
                    tensor(-0.2627), tensor(-0.3396), tensor(-0.6425)
                ], 
        'image_graph': array([
            [0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
            [0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ],
            [0.        , 0.08333334, 0.14285715, 0.14285715, 0.14285715, 0.08333334, 0.08333334, 0.08333334, 0.        ],
           ...
            [0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.        ]], 
            dtype=float32), 
        'flags': [
            False, False, False, False, False, 
            False, False, False, False, False, 
            False, False, False, False, False, 
            False, False, False, False, False, 
            False, False, False, False, False, 
            False, False, False, False, False, 
            False, False, False, False, False, 
            False, False, False, False, False, 
            False, False], 
        'flag': True, 
        'word': [], 
        'box_text': [
            ('black', tensor([-0.2899,  0.1239, -0.4740,  ..., -0.4807, -0.4082, -0.6435])), 
            ('white', tensor([-0.2732,  0.4018, -0.4781,  ..., -0.4782, -0.4708, -0.2510])), 
            ('white', tensor([-0.3721,  0.1859, -0.5632,  ..., -0.5610, -0.2828, -0.3253])), 
            ('white', tensor([-0.2054,  0.3215, -0.4104,  ..., -0.4107, -0.2413, -0.2379])), 
            ('blue', tensor([-0.2627,  0.2279, -0.4667,  ..., -0.4689, -0.3917,  0.1752])), 
            ('black', tensor([-0.3396,  0.0095, -0.5434,  ..., -0.5445, -0.7224,  0.1304])), 
            ('black', tensor([-0.6425, -0.8791, -0.8490,  ..., -0.8507, -0.6838, -0.2024]))
        ]
    }, 
    '691915808591712257': {
        'text': 'yep those penalty rates are a killer . ',
        ...
        'word': [], 
        'box_text': [
            ('blue', tensor([-0.0349,  0.5834, -0.2572,  ..., -0.2592, -0.4214,  0.2335])), 
            ('', tensor([-0.5933,  0.4554, -0.7719,  ..., -0.7807,  0.1455, -0.3950])), 
            ('blue', tensor([-0.0630,  0.7266, -0.2437,  ..., -0.2503, -0.1166, -0.1800])), 
            ('', tensor([-0.2423,  0.6497, -0.4318,  ..., -0.4427, -0.1593, -0.4385])),
            ('glass', tensor([-0.0430, -0.5507, -0.2726,  ..., -0.2768, -0.8849, -0.3752])), 
            ('black', tensor([-0.2212,  0.7649, -0.4232,  ..., -0.4304, -0.3663,  0.0212]))
        ]
    }
}
```